In [1]:
%%classpath add mvn 
org.scalanlp breeze_2.11 0.13.2
org.scalanlp breeze-natives_2.11 0.13.2
org.scalanlp breeze-viz_2.11 0.13.2

In [2]:
%classpath add jar jars/enets_2.11-1.0.jar

In [3]:
scala.util.Properties.versionString

version 2.11.12

In [4]:
import breeze.linalg._
import breeze.numerics._
import breeze.stats._
import breeze.numerics._
import breeze.stats.distributions.Gaussian
import breeze.optimize.{LBFGSB, LBFGS,StochasticGradientDescent, ApproximateGradientFunction, DiffFunction}

//https://github.com/scalanlp/breeze/pull/633

import breeze.linalg._
import breeze.numerics._
import breeze.stats._
import breeze.numerics._
import breeze.stats.distributions.Gaussian
import breeze.optimize.{LBFGSB, LBFGS, StochasticGradientDescent, ApproximateGradientFunction, DiffFunction}


In [5]:
type RealVector = DenseVector[Double]
type Features = DenseMatrix[Double]
type Target = DenseVector[Double]

defined type alias RealVector
defined type alias Features
defined type alias Target


In [6]:
val x = breeze.linalg.csvread(new java.io.File("xout.csv"))
val y = breeze.linalg.csvread(new java.io.File("yout.csv")).toDenseVector
val xs = DenseMatrix.horzcat(x,DenseMatrix.ones[Double](x.rows,1))
OutputCell.HIDDEN

In [7]:
import com.github.timsetsfire.enets.RlmNet
import com.github.timsetsfire.enets.robust.norms.HuberT
val rlm = new RlmNet(x,y, standardizeTarget =true, standardizeFeatures=true,rnorm=HuberT())
rlm.fit

Oct 18, 2018 12:21:06 PM com.github.fommil.netlib.LAPACK <clinit>
Oct 18, 2018 12:21:06 PM com.github.fommil.jni.JniLoader liberalLoad
INFO: successfully loaded C:\Users\WHITTA~1\AppData\Local\Temp\jniloader3283675447525051930netlib-native_ref-win-x86_64.dll
Oct 18, 2018 12:21:06 PM com.github.fommil.netlib.BLAS <clinit>
Oct 18, 2018 12:21:06 PM com.github.fommil.jni.JniLoader load
INFO: already loaded netlib-native_ref-win-x86_64.dll


null

In [9]:
val plot = new Plot
for (i <- 0 until x.cols) {
        plot.add(new Line { x = 1 to 100; y = rlm.b(i,::).t.toArray })
}
plot

In [10]:
def mad(x: RealVector,c: Double = Gaussian(0,1).inverseCdf(3/4d),center: (RealVector => Double) = (x: RealVector) => 0d) = {
  val m = center(x)
  median( abs( x - m) / c)
}

mad: (x: RealVector, c: Double, center: RealVector => Double)Double


In [66]:
class HuberRegressor(epsilon: Double = 1.345, alpha: Double = 1d, maxIters:Int = 100, tolerance:Double = 1e-4) { 
    
    private val epsSquare = math.pow(epsilon,2)
    
    def loss(x: Features, y: Target)(b: RealVector) = {
        val sigma = b(-1)
        val intercept = b(-2)
        val nSamples = x.rows
        val r = ( y - x*b(0 to -3)).map{ _ - b(-2)}
        sum(r.map{ 
            elem => 
            if( abs(elem) < epsilon*sigma) {
                math.pow(elem,2)/sigma
            } else {
                2*epsilon*abs(elem) - epsSquare*sigma
            }}) + sigma*nSamples
    }

    def lossGrad(x: Features, y: Target)(b: RealVector) = { 
        val sigma = b(-1)
        val intercept = b(-2)
        val nSamples = x.rows.toDouble
        val temp = DenseVector.zeros[Double](b.length)
        val r = ( y - x*b(0 to -3)).map{ _ - b(-2)}
        val psi = r.map{ elem => if( abs(elem) < epsilon*sigma) 2*elem/sigma else 2*epsilon*signum(elem)}
        temp(0 to -3) := (-x.t * psi).map{ _ / nSamples.toDouble}
        temp(-2) = -mean(psi) 
        temp(-1) = (x.rows - sum(r.map{ elem => if( abs(elem) < epsilon*sigma) math.pow(elem/sigma,2) else epsilon*epsilon }))/nSamples.toDouble
        temp
    }
    
    def maximumLikelihood(x: Features, y: Target, init: RealVector): RealVector = {
        val diffFunction = new DiffFunction[RealVector] {
            override def calculate(w: RealVector): (Double, RealVector) = (loss(x,y)(init), lossGrad(x,y)(init))
        }
        
        val autograd = new ApproximateGradientFunction(loss(x,y))
        
        val lower: DenseVector[Double] = DenseVector.fill(init.length){-Inf}
        lower(-1) = 1e-3
        val upper: DenseVector[Double] = DenseVector.fill(init.length){Inf}
             
        val lbfgs = new LBFGSB(lower, upper, m=pow(init.length,2)/2, maxIter=maxIters, tolerance = tolerance)
        lbfgs.minimize(autograd, init)
    }
       
}



defined class HuberRegressor


In [67]:
import breeze.stats._
// x(0, ::) 
val xs = x(*, ::) / stddev(x(::, *)).t
val ys = y / stddev(y)
OutputCell.HIDDEN

stddev(xs(::, *))

val hr = new HuberRegressor()
// val q = inv(xs.t * xs) * xs.t * y
val q = DenseVector.zeros[Double](x.cols + 2)
q(-1) = 1

q := hr.maximumLikelihood(xs,y,q)
println(hr.loss(x,y)(q))
println(q)

587106.1745260417
DenseVector(-1.0320324412307829, 0.7987108549197264, 0.005406138728977784, 0.3973413985715523, -1.0596624608020682, 3.6063955670812926, -0.7507035695814653, -2.1883290546686305, 1.5977129032133814, -1.7669662468548923, -1.5937921776630397, 1.0428716125043005, -2.3052048326161074, 16.41302450887255, 1.9865378858301952)


null

In [68]:
case class HuberT(k: Double = 1.345) {
  def rho(z: RealVector) = z.map{ elem => if( abs(elem) < k) pow(elem,2) else 2*k*abs(elem) - k*k }
  def psi(z: RealVector) = z.map{ elem => if( abs(elem) < k) 2*elem else 2*k*signum(elem)}
  def psiDeriv(z: RealVector) = z.map{ elem => if( abs(elem) < k) 1d else 0d}
  def w(z: RealVector) = z.map{ elem => if( abs(elem) < k) 1 else k / abs (elem) }
}
def mad(x: RealVector,c: Double = Gaussian(0,1).inverseCdf(3/4d),center: (RealVector => Double) = (x: RealVector) => 0d) = {
  val m = center(x)
  median( abs( x - m) / c)
}

class IRLS(epsilon: Double = 1.345, alpha: Double = 1d, maxIters:Int=10) { 
 
    val norm = HuberT()
    
    def loss(x: Features, y: Target, weight: RealVector)(b: RealVector) = {
        val nSamples = x.rows
        val r = ( y - x*b(0 to -2)).map{ _ - b(-1)}
        mean( weight *:* r *:* r)
    }
    def lossGrad(x: Features, y: Target, weight: RealVector)(b: RealVector) = { 
        val nSamples = x.rows
        val r = ( y - x*b(0 to -2)).map{ _ - b(-1)}
        val temp = DenseVector.zeros[Double](b.length)
        temp(0 to -2) := (x.t * (weight *:* r)).map{ _ / -x.rows.toDouble }
        temp(-1) = -1*mean(weight *:* r)
        temp
    }
        
    def maximumLikelihood(x: Features, y: Target, init: RealVector): RealVector = {
        
        val r = (y - x * init(0 to -2)).map{ _ - init(-1)}
        val weight = sqrt(norm.w(r /mad(r)))
        val xs = if(init.length == x.cols) x else DenseMatrix.horzcat(x, DenseMatrix.ones[Double](x.rows, 1))
        val wexog =  xs(::, *) * weight
             
        val diffFunction = new DiffFunction[RealVector] {
            override def calculate(w: RealVector): (Double, RealVector) = (loss(xs,y,weight)(w), lossGrad(xs,y,weight)(w))
        }
                   
        for(i <- 0 to maxIters) { 
            val r = (y - xs * init(0 to -1))
            weight := sqrt(norm.w(r / mad(r)))
//             init := inv( (xs(::,*)*weight).t * xs ) * xs.t * (weight *:* y)
            init := (xs(::, *) * weight) \ (y *:* weight)
        }
        init
    }
}

defined class HuberT
mad: (x: RealVector, c: Double, center: RealVector => Double)Double
defined class IRLS


In [81]:
val x = DenseMatrix.rand(1000,10, Gaussian(0,1))
val y = DenseVector.rand(1000, Gaussian(0,1))
OutputCell.HIDDEN

In [82]:
val hr = new IRLS(maxIters=10)
val q = DenseVector.zeros[Double](x.cols + 1)
OutputCell.HIDDEN

In [83]:
hr.maximumLikelihood(x,y,q)

DenseVector(-0.0022477518702361046, -0.018730997701338778, 0.03011015647674628, 0.02056878287202675, 0.03670674006897167, -0.020815522726142, -0.0039395040036241235, -0.014959433027974735, -0.00888533826768804, 0.022574464978395135, -0.08904871898126585)

In [84]:
import breeze.linalg.qr

import breeze.linalg.qr


In [88]:
val x = DenseMatrix.rand(1000,100, Gaussian(0,1))
val y = DenseVector.rand(1000, Gaussian(0,1))
OutputCell.HIDDEN
val rlm = new RlmNet(x,y,rnorm = com.github.timsetsfire.enets.robust.norms.HuberT())

com.github.timsetsfire.enets.RlmNet@3bf68335

In [89]:
rlm.fit

null

In [90]:
val plot = new Plot
for (i <- 0 until x.cols) {
        plot.add(new Line { x = 1 to 100; y = rlm.b(i,::).t.toArray })
}
plot

(y - xb)T W (y - xb)  + lambda ||b||2

-x(y-xb)W + 2*b
-xTWy + xTWxb + 2b = 0
-xTWy = (xTWx + 2lambda)b

In [50]:
val t = x * inv(diag(stddev(xs(::, *))))

diag(stddev(x(::, *))) * inv(t.t * t) * t.t * y 

DenseVector(-0.7990535813415973, 1.136152252385928, -0.027852892818770937, 0.7248986942712289, -0.3323877394029593, 4.165218258260862, -0.20462349150181308, -2.0394100690094383, 1.4902572679630801, -1.5836110448673066, -0.8490719903222631, 1.360805664346451, -2.972855832986659)

In [16]:
val i = qr( x )
i.r.cols

13

In [17]:
val weights = DenseVector.ones[Double](y.length)
val endog = y
val exog = DenseMatrix.horzcat(x , DenseMatrix.ones[Double](x.rows, 1))

val wHalf = sqrt(weights)
val wendog = endog *:* weights
val wexog = exog(::, *).map{ elem => elem*:* wHalf}
OutputCell.HIDDEN

0.00632  18.0   2.31   0.0  0.538   6.575  65.2   4.09    1.0  ... (14 total)
0.02731  0.0    7.07   0.0  0.469   6.421  78.9   4.9671  2.0  ...
0.02729  0.0    7.07   0.0  0.469   7.185  61.1   4.9671  2.0  ...
0.03237  0.0    2.18   0.0  0.458   6.998  45.8   6.0622  3.0  ...
0.06905  0.0    2.18   0.0  0.458   7.147  54.2   6.0622  3.0  ...
0.02985  0.0    2.18   0.0  0.458   6.43   58.7   6.0622  3.0  ...
0.08829  12.5   7.87   0.0  0.524   6.012  66.6   5.5605  5.0  ...
0.14455  12.5   7.87   0.0  0.524   6.172  96.1   5.9505  5.0  ...
0.21124  12.5   7.87   0.0  0.524   5.631  100.0  6.0821  5.0  ...
0.17004  12.5   7.87   0.0  0.524   6.004  85.9   6.5921  5.0  ...
0.22489  12.5   7.87   0.0  0.524   6.377  94.3   6.3467  5.0  ...
0.11747  12.5   7.87   0.0  0.524   6.009  82.9   6.2267  5.0  ...
0.09378  12.5   7.87   0.0  0.524   5.889  39.0   5.4509  5.0  ...
0.62976  0.0    8.14   0.0  0.538   5.949  61.8   4.7075  4.0  ...
0.63796  0.0    8.14   0.0  0.538   6.096  84.5   4

In [36]:
exog(::, *) * wHalf == exog(::, *).map{ elem => elem*:* wHalf}

true

In [19]:
// http://www.math.iit.edu/~fass/477577_Chapter_5.pdf

input is incomplete: input is incomplete

In [57]:

val xtx = wexog.t * wexog
val xty = wexog.t * wendog
val c = cholesky(xtx + DenseMatrix.eye[Double](wexog.cols)*10d)
val w = c \ xty
c.t \ w

DenseVector(-0.09383025053527433, 0.05095982087221412, -0.016363368562327035, 2.145110781983357, -0.5019802061934725, 5.451399976856914, -0.006285366818846987, -0.9592717600484365, 0.18578045607241872, -0.010436638671028065, -0.4127724656574354, 0.014629513832620306, -0.45437374359116783, 2.9930858386819863)

In [ ]:
DenseMatrix.eye()

In [32]:
val QR = qr(wexog)
OutputCell.HIDDEN

QR(-3.013957672193257E-5   0.03086854015653992     ... (506 total)
-1.3023921523367635E-4  -4.992343632689267E-7   ...
-1.3014383682632836E-4  -4.988687576755804E-7   ...
-1.5436995229271708E-4  -5.917325645276132E-7   ...
-3.2929395136892535E-4  -1.262253122663938E-6   ...
-1.423522729668707E-4   -5.45666266640384E-7    ...
-4.210479792376889E-4   0.021434952484272768    ...
-6.893474391075766E-4   0.021433924035892157    ...
-0.0010073867384094396  0.02143270492422307     ...
-8.10907219272586E-4    0.021433458071632973    ...
-0.0010724825014244406  0.021432455398442848    ...
-5.602050755584021E-4   0.021434419065791674    ...
-4.472293520547114E-4   0.02143485212555237     ...
-0.003003275290573417   -1.1512187205341663E-5  ...
-0.0030423804375860914  -1.1662085476244543E-5  ...
-0.0029919729493026806  -1.1468862948995335E-5  ...
-0.0050261082428132015  -1.9266132274716938E-5  ...
-0.0037397873521145732  -1.433539317585894E-5   ...
-0.003828060068115135   -1.467376110200679E-5   .

In [33]:
%%timeit
QR.r \ (QR.q.t * wendog)

69 ms ± 17 ms per loop (mean ± std. dev. of 3 run, 10 loop each)


In [30]:
%%timeit
wexog \ wendog

52 ms ± 8 ms per loop (mean ± std. dev. of 3 run, 10 loop each)


In [31]:
%%timeit
inv(wexog.t * wexog) * wexog.t * wendog

65 ms ± 3 ms per loop (mean ± std. dev. of 3 run, 10 loop each)


In [28]:
exog \ endog

DenseVector(-0.10801135783679901, 0.04642045836688059, 0.02055862636706894, 2.686733819344869, -17.766611228299915, 3.809865206809225, 6.92224640344334E-4, -1.4755668456002504, 0.3060494789851739, -0.012334593916574477, -0.9527472317072868, 0.00931168327379386, -0.5247583778554888, 36.45948838508974)

In [29]:
%%timeit
LSMR.solve(exog, endog)

84 ms ± 5 ms per loop (mean ± std. dev. of 3 run, 10 loop each)


In [61]:
self.endog = endog
self.e`xog = exog
self.weights = weights
w_half = np.sqrt(weights)

self.wendog = w_half * endog
if np.isscalar(weights):
    self.wexog = w_half * exog
else:
    self.wexog = w_half[:, None] * exog
Q, R = np.linalg.qr(self.wexog)
params = np.linalg.solve(R, np.dot(Q.T, self.wendog))

70 ms ± 54 ms per loop (mean ± std. dev. of 3 run, 10 loop each)


In [ ]:
class _MinimalWLS(object):
    """
    Minimal implementation of WLS optimized for performance.
    Parameters
    ----------
    endog : array-like
        1-d endogenous response variable. The dependent variable.
    exog : array-like
        A nobs x k array where `nobs` is the number of observations and `k`
        is the number of regressors. An intercept is not included by default
        and should be added by the user. See
        :func:`statsmodels.tools.add_constant`.
    weights : array-like, optional
        1d array of weights.  If you supply 1/W then the variables are pre-
        multiplied by 1/sqrt(W).  If no weights are supplied the default value
        is 1 and WLS reults are the same as OLS.
    Notes
    -----
    Need resid, scale, fittedvalues, model.weights!
        history['scale'].append(tmp_results.scale)
        if conv == 'dev':
            history['deviance'].append(self.deviance(tmp_results))
        elif conv == 'sresid':
            history['sresid'].append(tmp_results.resid/tmp_results.scale)
        elif conv == 'weights':
            history['weights'].append(tmp_results.model.weights)
    Does not perform and checks on the input data
    """

    def __init__(self, endog, exog, weights=1.0):
        self.endog = endog
        self.exog = exog
        self.weights = weights
        w_half = np.sqrt(weights)

        self.wendog = w_half * endog
        if np.isscalar(weights):
            self.wexog = w_half * exog
        else:
            self.wexog = w_half[:, None] * exog

    def fit(self, method='pinv'):
        """
        Minimal implementation of WLS optimized for performance.
        Parameters
        ----------
        method : str, optional
            Method to use to estimate parameters.  "pinv", "qr" or "lstsq"
              * "pinv" uses the Moore-Penrose pseudoinverse
                 to solve the least squares problem.
              * "qr" uses the QR factorization.
              * "lstsq" uses the least squares implementation in numpy.linalg
        Returns
        -------
        results : namedtuple
            Named tuple containing the fewest terms needed to implement
            iterative estimation in models. Currently
              * params : Estimated parameters
              * fittedvalues : Fit values using original data
              * resid : Residuals using original data
              * model : namedtuple with one field, weights
              * scale : scale computed using weighted residuals
        Notes
        -----
        Does not perform and checks on the input data
        See Also
        --------
        statsmodels.regression.linear_model.WLS
        """
        if method == 'pinv':
            pinv_wexog = np.linalg.pinv(self.wexog)
            params = pinv_wexog.dot(self.wendog)
        elif method == 'qr':
            Q, R = np.linalg.qr(self.wexog)
            params = np.linalg.solve(R, np.dot(Q.T, self.wendog))
        else:
            params, _, _, _ = np.linalg.lstsq(self.wexog, self.wendog,
                                              rcond=-1)

        fitted_values = self.exog.dot(params)
        resid = self.endog - fitted_values
        wresid = self.wendog - self.wexog.dot(params)
        df_resid = self.wexog.shape[0] - self.wexog.shape[1]
        scale = np.dot(wresid, wresid) / df_resid

        return Bunch(params=params, fittedvalues=fitted_values, resid=resid,
                     model=self, scale=scale)

In [ ]:
import breeze.stats._
val xs = DenseMatrix.horzcat(x, DenseMatrix.ones[Double](x.rows, 1))

val r = (y - xs * q)
val weight = HuberT().w(r / mad(r))




for(i <- 0 to 10) { 
    r := (y - xs *q )
    weight := HuberT().w(r / mad(r))
    q := inv( (xs(::,*)*weight).t * xs ) * xs.t * (weight *:* y)
    println(q(-1))
}



In [ ]:
q

In [ ]:
println(q)

In [ ]:
val ag = new ApproximateGradientFunction(hr.loss(xs,y))

In [ ]:
ag.calculate(q)._2

In [ ]:
hr.lossGrad(x,y)(q)

In [ ]:
hr.lossGrad(x,y)(qSklearn) - ag.calculate(qSklearn)._2

In [ ]:
val qSklearn = DenseVector(-0.11828755,  0.02990455, -0.02083657,  1.83264527, -0.33127537,  6.16875147,
 -0.038073,   -0.8278615 ,  0.13821192, -0.01013945, -0.56633751,  0.01356108,
 -0.27942803, 1.0998216367119165, 1.9464638350773724)
hr.loss(x,y)(qSklearn)

In [ ]:
val norm = HuberT()

def loss(x: Features, y: Target, weight: RealVector)(b: RealVector) = {
    val nSamples = x.rows
    val r = ( y - x*b(0 to -2)).map{ _ - b(-1)}
    mean( weight *:* r *:* r)
}
def lossGrad(x: Features, y: Target, weight: RealVector)(b: RealVector) = { 
    val nSamples = x.rows
    val r = ( y - x*b(0 to -2)).map{ _ - b(-1)}
    val temp = DenseVector.zeros[Double](b.length)
    temp(0 to -2) := (x.t * (weight *:* r)).map{ _ / -x.rows.toDouble }
    temp(-1) = -1*mean(weight *:* r)
    temp
}
val weight = DenseVector.ones[Double](y.length)
var c = mad(y)

val diffFunction = new DiffFunction[RealVector] {
    override def calculate(w: RealVector): (Double, RealVector) = (loss(x,y,weight)(w), lossGrad(x,y,weight)(w))
}

In [ ]:
diffFunction.calculate(b)._2

In [ ]:
val hr = new HuberRegressor()
val q = DenseVector.zeros[Double](x.cols + 1)
q(-1) = 1.0
hr.loss(x,y)(q)

In [ ]:
q := hr.maximumLikelihood(x,y,q)

In [ ]:
hr.loss(x,y)(q)

In [ ]:
val b = q.copy

In [ ]:
val epsilon = 1.345
val sigma = b(-1)
val intercept = b(-2)
val nSamples = x.rows
val r = ( y - x*b(0 to -3)).map{ _ - b(-2)}
println("square", sum(r.map{ elem => if( abs(elem) < epsilon*sigma) math.pow(elem,2)/sigma else 0}) )
println("outlier", sum(r.map{ elem => if( abs(elem) < epsilon*sigma) 0 else 2*epsilon*abs(elem/sigma)*sigma - epsilon*epsilon*sigma}) )
println(x.rows)
println(sigma)

In [ ]:
q

In [ ]:
(hr.loss(x,y)(q), hr.lossGrad(x,y)(q))

In [ ]:
val autograd = new ApproximateGradientFunction(hr.loss(x,y))

In [ ]:
autograd.calculate(q)._2

In [ ]:
val b = q.copy     
val sigma = b(-1)
val intercept = b(-2)
val nSamples = x.rows
val r = ( y - x*b(0 to -3)).map{ _ - b(-2)}) 

println((sigma, intercept, nSamples))

sum(r.map{ elem => if( abs(elem) < sigma*1.345) math.pow(elem/sigma,2) else 0d})

In [ ]:
b

In [ ]:
hr.lossGrad(x,y)(q)

In [ ]:
val w = q.copy
//costGrad(x,y)(w)


In [ ]:
val sigma = w(-1)
val intercept = w(-2)
val nSamples = x.rows
val epsilon = 1.3456

val k = 1.345
val temp = DenseVector.zeros[Double](w.length)

val r = ( y - x*w(0 to -3).map{ _ + w(-2)}) 




val psi = r.map{ elem => if( abs(elem) < k) 2*elem else 2*k*signum(elem)}
temp(0 to -3) := -x.t * psi
temp(-2) = -sum(psi)
temp(-1) = x.rows - sum(r.map{ elem => if( abs(elem) < k*sigma) math.pow(elem/sigma,2) else k*k })
temp

In [ ]:
sum(r.map{ elem => if( abs(elem) < k*sigma) math.pow(elem, 2)/sigma else (2*k*abs(elem/sigma) - k*k)*sigma}) 

In [ ]:
sum(r.map{ elem => if( abs(elem) < k*sigma) elem*elem/sigma else 2*k*abs(elem/sigma)*sigma - k*k*sigma}) + 506*sigma

In [ ]:
sum(r.map{ elem => if( abs(elem) < k*sigma) 0 else 2*k*abs(elem/sigma)*sigma - k*k*sigma})

In [ ]:
sum(r.map{ elem => if( abs(elem) < k*sigma) 0d else k*k})

In [ ]:
nSamples - sum(r.map{ elem => if( abs(elem) < k*sigma) 0d else k*k}) + sum(r.map{ elem => if( abs(elem) < k*sigma) elem*elem else 0d})/sigma

In [ ]:

val g = new ApproximateGradientFunction(cost(xs,y))

In [ ]:
val upper = DenseVector.fill(x.cols + 2){Inf}
val lower = DenseVector.fill(x.cols + 2){-Inf}
lower(-1) = 1e-6

In [ ]:
val lbfgsb = new LBFGSB(lower, upper, maxIter=1000)
lbfgsb.minimize(g, w)

In [ ]:
case class HuberT(k: Double = 1.345) {
  def rho(z: RealVector) = z.map{ elem => if( abs(elem) < k) pow(elem,2) else 2*k*abs(elem) - k*k }
  def psi(z: RealVector) = z.map{ elem => if( abs(elem) < k) 2*elem else 2*k*signum(elem)}
  def psiDeriv(z: RealVector) = z.map{ elem => if( abs(elem) < k) 1d else 0d}
  def w(z: RealVector) = z.map{ elem => if( abs(elem) < k) 1 else k / abs (elem) }
}
def mad(x: RealVector,c: Double = Gaussian(0,1).inverseCdf(3/4d),center: (RealVector => Double) = (x: RealVector) => 0d) = {
  val m = center(x)
  median( abs( x - m) / c)
}

In [ ]:
val huber = HuberT()

-xs.t * huber.psi( y - xs*w(0 to -2))

In [ ]:
val r = y - xs*w(0 to -2)
sum(huber.rho( r )) + 1.0*xs.rows

-1*sum(huber.psi(r))*2 + xs.rows

In [ ]:

val linearLoss = y - xs*init
OutputCell.HIDDEN

In [ ]:
val absLinearLoss = abs(linearLoss)
OutputCell.HIDDEN

In [ ]:
outlierMask.activeSize

In [ ]:
val w = DenseVector.zeros[Double](x.cols + 2)
w(-1) = 1d


In [ ]:
// val b = DenseVector.zeros[Double](xs.cols + 1)
// b(-1) = 1d
// val x = xs.copy


// val sigma = b(-1)
// val intercept = b(-2)
// val nSamples = x.rows
// //
// val epsilon = 1.3456
// val sampleWeight = DenseVector.ones[Double](y.length)
// val linearLoss = y - x*b(0 to -2)
// //
// val nFeatures = x.cols

// val outlierMask = BitVector(absLinearLoss.map{_ > 1.345*sigma}.toArray:_*)
// val outliers = absLinearLoss(outlierMask)

// val numOutliers = outlierMask.activeSize
// val outliersSw = sampleWeight(outlierMask)
// val nSwOutliers = sum(outliersSw)
// val outlierLoss = (2.0 * epsilon * sum(outliersSw * outliers) - sigma * nSwOutliers * scala.math.pow(epsilon, 2))


// val nonOutliers = linearLoss(!outlierMask)
// val weightedNonOutliers = sampleWeight(!outlierMask) *:* nonOutliers
// val weightedLoss = weightedNonOutliers.t * nonOutliers
// val squaredLoss = weightedLoss / sigma

// nSamples * sigma + squaredLoss + outlierLoss

In [ ]:
cost(x,y)(b)

In [ ]:
import breeze.optimize.{LBFGS, ApproximateGradientFunction, DiffFunction}
val g = new ApproximateGradientFunction(cost(x,y))

In [ ]:
g(b)

In [ ]:
val sigma = w(-1)
val intercept = w(-2)
val nFeatures = x.cols
val epsilon = 1.345
val sampleWeight = DenseVector.ones[Double](y.length)


val linearLoss = y - xs*w(0 to -2)
val absLinearLoss = abs(linearLoss)
val outlierMask = BitVector(absLinearLoss.map{_ > 1.345*sigma}.toArray:_*)
val outliers = absLinearLoss(outlierMask)

val numOutliers = outlierMask.activeSize
val outliersSw = sampleWeight(outlierMask)
val nSwOutliers = sum(outliersSw)
val outlierLoss = (2.0 * epsilon * sum(outliersSw * outliers) - sigma * nSwOutliers * scala.math.pow(epsilon, 2))

val nonOutliers = linearLoss(!outlierMask)

In [ ]:
sum(outliersSw * outliers) - sigma * nSwOutliers * scala.math.pow(epsilon, 2)

In [ ]:
println(sum(outliersSw * outliers))

println(sigma)
println(nSwOutliers)
println(scala.math.pow(epsilon, 2))

In [ ]:
def g(z: RealVector, c: Double, k: Double = 1.345) = z.map{ elem => if( abs(elem) < 1.345*c) pow(elem,2) else 2*k*abs(elem) - k*k }

In [ ]:
sum(g(r, 1d))

In [ ]:
sum(HuberT().rho(r))

In [ ]:
-xs.t * HuberT().psi(r)

In [ ]:
sum(HuberT().psi(r/200d) *:* (r/25d)) * -1 

In [ ]:
val weight = DenseVector.ones[Double](xs.rows)
val resid = y.copy
def g(xs: Features, y: Target, weight: RealVector)(b: RealVector) = { 
    val r = y - xs * b    
    sum(weight *:* r *:* r ) / r.length.toDouble
}

In [ ]:
(xs.t * HuberT(1.35).psi(r)).map{ _ * -1d}

In [ ]:
HuberT()

In [ ]:
val cost = g(xs, y, weight)_

In [ ]:
val diffg = new ApproximateGradientFunction(cost)

In [ ]:
val lbfgs = new LBFGS[DenseVector[Double]](m=pow(init.length,2)/2,tolerance = 1e-4)

In [ ]:

// val diffFunction = new DiffFunction[RealVector] {
//   override def calculate(w: RealVector): (Double, RealVector) =
//     (loss(w, x, y), lossGrad(w, x, y))
// }
for(i <- 0 to 10) {
    init := lbfgs.minimize(diffg, init)
    resid := y - xs * init
    weight := HuberT().rho( resid / mad(resid))
    println(init(0))
}

In [ ]:
inv(xs.t * xs) * xs.t * y 